In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
import xarray as xr
import hvplot.xarray

In [ ]:
import xarray as xr
import xrspatial as xrs
import datashader as ds
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
import geoviews as gv

In [ ]:
data = xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif")

In [ ]:
forxr = data.where(data>200)[0,:,:]

In [ ]:
forxr

In [ ]:
test = xr.DataArray([[i for i in range(300)],[i for i in range(300)],[ i for i in range(300)]])

In [ ]:
curve = xrs.curvature(forxr)

In [ ]:
shade(curve)

In [ ]:
hey=xr.DataArray([1,2,3,4])

In [ ]:
hey.data

In [ ]:
hillshaded = xrs.hillshade(forxr)

In [ ]:
slope = xrs.slope(forxr)
#slope = slope.where(np.logical_and(slope.data > 25, slope.data < 50))
slope = slope.where(slope.data>50)

In [ ]:
slope.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = 'inferno', alpha = 20)

In [ ]:
shade(hillshaded, cmap = ['black','white'], )